In [1]:
import pandas as pd


In [ ]:
votacao = pd.read_excel('../TabelaVotacaoFull.xlsx')
presenca = pd.read_excel('..TabelaPresencaFull.xlsx')

In [ ]:
votacao.head()

In [ ]:
votacao.rename(columns={'CPF Full': 'CPF'}, inplace=True)

# Remover os caracteres '-' e '/' da coluna 'CPF Full'
votacao['CPF'] = votacao['CPF'].str.replace(r'[-/,.]', '', regex=True)
votacao['Nome'] = votacao['Nome'].str.replace(r'\|', 'I', regex=True)

In [ ]:
presenca.head()

In [ ]:
presenca.rename(columns={'Documento': 'CPF'}, inplace=True)

# Remover os caracteres '-' e '/' da coluna 'CPF Full'
presenca['CPF'] = presenca['CPF'].str.replace(r'[-/,.]', '', regex=True)
presenca['Nome'] = presenca['Nome'].str.replace(r'\|', 'I', regex=True)

In [ ]:
from rapidfuzz import fuzz, process

In [ ]:
def fuzzy_join(df1, df2, key1, key2, threshold=90):
    matches = []
    for idx, value in df1[key1].items():
        # Encontrar a melhor correspondência para cada valor
        best_match = process.extractOne(value, df2[key2], scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= threshold:  # Similaridade acima do limite
            matched_idx = df2[df2[key2] == best_match[0]].index[0]
            matches.append((idx, matched_idx))
    return matches

# Realizar o join com base na similaridade de nomes
matches = fuzzy_join(presenca, votacao, key1='Nome', key2='Nome', threshold=90)

# Unir os DataFrames com base nas correspondências
presenca['match_index'] = None
for pres_idx, vot_idx in matches:
    presenca.loc[pres_idx, 'match_index'] = vot_idx

result = presenca.merge(votacao, left_on='match_index', right_index=True, how='left')
result.drop(columns=['match_index'], inplace=True)

# Exibir o resultado
print(result)

In [ ]:
result.to_excel("resultado_limpo.xlsx", index=False)